<a href="https://colab.research.google.com/github/lr0103/data-science/blob/master/MapasCom4SquareAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Working with Maps and 4 Square API**

In this notebook we will see about:


*   4 Square API
*   Maps API
*   Scraping HTML pages



In [9]:
#Importação das libraries
import pandas as pd

###**1 - Data Scraping from HTML Page**

####1.1 -Making HTML page download

In [37]:
!wget -q -O 'canadatapage.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


####1.2 - HTML page Transformation on Pandas's Data Frame

In [54]:
df_html = pd.read_html('canadatapage.html')
df_html[0]

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [57]:
df_canada_raw=df_html[0]

In [58]:
df_canada_raw.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


####1.4 - Cleaning dataset

In [59]:
#Ignore cells with a borough that is Not assigned
df_canada= df_canada_raw[ df_canada_raw.Borough != 'Not assigned']

In [60]:
df_canada

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [61]:
#If a cell has a borough but a Not assigned neighborhood, 
# then the neighborhood will be the same as the borough
df_canada.loc[df_canada['Neighbourhood']== 'Not assigned', 'Borough']= df_canada['Borough'] 


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [65]:
print('Canda dataframe shape:',df_canada.shape)

Canda dataframe shape: (103, 3)


In [1]:
pip install geopy